In [2]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.mllib.clustering import *
import os
from hdfs import *
import csv
from pyspark.conf import SparkConf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark import SQLContext
from pyspark.context import SparkContext

In [3]:
lines = []
with open(r'Bowl.csv') as file:
    reader = csv.reader(file,delimiter=',')
    next(reader)
    for row in reader:
        row = list(map(lambda x: x.rstrip('*') if '*' in x else x,row))
        row = list(map(lambda x: '0.0' if '-' in x else x,row))
        row = list(map(lambda x: ' '.join(x.split('/')) if '/' in x else x,row))
        row = list(map(lambda x:x if ' ' in x or x == '-' else float(x),row))
        #print(row)
        lines.append(row)
    print(lines)
        #print(len(row))
        #print(row)
        

[['S Ladda', 1.0, 1.0, 2.0, 0.0, 21.0, 0.0, 0.0, 0.0, 10.5, 0.0, 0.0, 0.0, 0.0, 0.0], ['VR Aaron', 8.0, 8.0, 23.4, 0.0, 227.0, 2.0, '2 37', 113.5, 9.59, 71.0, 0.0, 0.0, 0.0, 0.0], ['KJ Abbott', 5.0, 5.0, 16.0, 0.0, 177.0, 2.0, '1 38', 88.5, 11.06, 48.0, 0.0, 0.0, 0.0, 0.0], ['AD Nath', 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ['MA Agarwal', 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ['HM Amla', 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ['Ankit Sharma', 3.0, 3.0, 11.0, 0.0, 84.0, 1.0, '1 27', 84.0, 7.63, 66.0, 0.0, 0.0, 1.0, 0.0], ['Anureet Singh', 2.0, 2.0, 6.0, 0.0, 43.0, 0.0, 0.0, 0.0, 7.16, 0.0, 0.0, 0.0, 0.0, 0.0], ['S Aravind', 9.0, 9.0, 29.1, 0.0, 216.0, 11.0, '2 18', 19.63, 7.4, 15.9, 0.0, 0.0, 3.0, 0.0], ['A Ashish Reddy', 4.0, 2.0, 3.5, 0.0, 39.0, 1.0, '1 14', 39.0, 10.17, 23.0, 0.0, 0.0, 1.0, 0.0], ['M Ashwin', 10.0, 10.0, 28.3, 0.0, 241.0, 7.0, '3 36', 34.42, 8.45, 24.4, 0.0, 0.0, 0.0, 0.0

In [13]:
lines2 = []
for i in (lines):
    lines2.append([i[8],i[9]])
    print(i)
print(lines2)

['S Ladda', 1.0, 1.0, 2.0, 0.0, 21.0, 0.0, 0.0, 0.0, 10.5, 0.0, 0.0, 0.0, 0.0, 0.0]
['VR Aaron', 8.0, 8.0, 23.4, 0.0, 227.0, 2.0, '2 37', 113.5, 9.59, 71.0, 0.0, 0.0, 0.0, 0.0]
['KJ Abbott', 5.0, 5.0, 16.0, 0.0, 177.0, 2.0, '1 38', 88.5, 11.06, 48.0, 0.0, 0.0, 0.0, 0.0]
['AD Nath', 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['MA Agarwal', 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['HM Amla', 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Ankit Sharma', 3.0, 3.0, 11.0, 0.0, 84.0, 1.0, '1 27', 84.0, 7.63, 66.0, 0.0, 0.0, 1.0, 0.0]
['Anureet Singh', 2.0, 2.0, 6.0, 0.0, 43.0, 0.0, 0.0, 0.0, 7.16, 0.0, 0.0, 0.0, 0.0, 0.0]
['S Aravind', 9.0, 9.0, 29.1, 0.0, 216.0, 11.0, '2 18', 19.63, 7.4, 15.9, 0.0, 0.0, 3.0, 0.0]
['A Ashish Reddy', 4.0, 2.0, 3.5, 0.0, 39.0, 1.0, '1 14', 39.0, 10.17, 23.0, 0.0, 0.0, 1.0, 0.0]
['M Ashwin', 10.0, 10.0, 28.3, 0.0, 241.0, 7.0, '3 36', 34.42, 8.45, 24.4, 0.0, 0.0, 0.0, 0.0]
['R Ashwi

In [5]:
#vecAssembler = VectorAssembler(inputCols=["avg","er"], outputCol="features")

In [6]:
final = []
for i in enumerate(lines2):
    final.append([i[1][0],i[1][1]])
print(final)

[[0.0, 10.5], [113.5, 9.59], [88.5, 11.06], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [84.0, 7.63], [0.0, 7.16], [19.63, 7.4], [39.0, 10.17], [34.42, 8.45], [31.9, 7.25], [0.0, 0.0], [0.0, 0.0], [40.66, 7.17], [0.0, 0.0], [134.0, 7.88], [40.33, 8.64], [27.0, 7.71], [39.0, 9.75], [27.0, 8.15], [29.05, 8.82], [27.06, 7.8], [0.0, 0.0], [53.0, 9.35], [19.09, 8.15], [0.0, 8.2], [0.0, 0.0], [24.81, 7.72], [38.0, 7.6], [16.0, 7.16], [0.0, 0.0], [0.0, 0.0], [28.0, 8.0], [0.0, 0.0], [0.0, 0.0], [21.0, 7.57], [27.5, 7.85], [0.0, 0.0], [0.0, 0.0], [83.5, 9.82], [0.0, 0.0], [0.0, 0.0], [31.5, 6.3], [0.0, 6.0], [0.0, 0.0], [15.5, 7.75], [0.0, 0.0], [43.55, 8.0], [18.5, 5.55], [0.0, 0.0], [34.58, 7.98], [35.5, 8.87], [51.5, 7.92], [32.33, 7.46], [27.6, 8.62], [35.5, 8.52], [0.0, 0.0], [0.0, 0.0], [38.87, 7.74], [0.0, 0.0], [89.0, 8.9], [53.0, 9.63], [23.45, 9.21], [0.0, 0.0], [32.0, 8.34], [0.0, 0.0], [33.7, 7.71], [0.0, 0.0], [0.0, 13.0], [16.66, 8.33], [20.22, 7.42], [21.3, 7.42], [44.55, 7.78], [0.0, 0

In [10]:
bowler_clusters = []
dataa = sc.parallelize(final)
model = KMeans.train(dataa, 8, maxIterations=10,initializationMode="random")
print(model)
for i in range(len(final)):
    print(model.predict(np.array(final[i])))
    bowler_clusters.append([lines[i][0],model.predict(np.array(final[i]))])

0
4
4
0
0
0
4
0
1
2
2
5
0
0
2
0
4
2
5
2
5
5
5
0
6
1
0
0
1
2
3
0
0
5
0
0
1
5
0
0
4
0
0
5
0
0
3
0
2
3
0
2
2
6
5
5
2
0
0
2
0
4
6
1
0
5
0
5
0
0
3
1
1
2
0
1
0
0
3
0
0
0
2
5
6
0
5
5
1
0
1
6
0
0
5
4
1
0
0
6
2
0
4
5
4
0
0
5
2
0
0
0
0
0
5
0
0
1
1
0
0
6
0
1
0
6
6
6
6
0
5
0
0
2
1
0
2
5
0
1
0
0
5
0
0
0
0
0
1
0
0
1
0
0
6
0
1
0
3


In [11]:
print(bowler_clusters)

[['S Ladda', 0], ['VR Aaron', 4], ['KJ Abbott', 4], ['AD Nath', 0], ['MA Agarwal', 0], ['HM Amla', 0], ['Ankit Sharma', 4], ['Anureet Singh', 0], ['S Aravind', 1], ['A Ashish Reddy', 2], ['M Ashwin', 2], ['R Ashwin', 5], ['GJ Bailey', 0], ['F Behardien', 0], ['R Bhatia', 2], ['SW Billings', 0], ['STR Binny', 4], ['Bipul Sharma', 2], ['SM Boland', 5], ['TA Boult', 2], ['CR Brathwaite', 5], ['DJ Bravo', 5], ['JJ Bumrah', 5], ['JC Buttler', 0], ['KC Cariappa', 6], ['YS Chahal', 1], ['DL Chahar', 0], ['UBT Chand', 0], ['PP Chawla', 1], [0.0, 2], ['BCJ Cutting', 3], ['Q de Kock', 0], ['AB de Villiers', 0], ['R Dhawan', 5], ['S Dhawan', 0], ['MS Dhoni', 0], ['AB Dinda', 1], ['JP Duminy', 5], ['F du Plessis', 0], ['ER Dwivedi', 0], ['JP Faulkner', 4], ['AJ Finch', 0], ['G Gambhir', 0], ['CH Gayle', 5], ['S Gopal', 0], ['MJ Guptill', 0], ['Gurkeerat Singh', 3], ['PSP Handscomb', 0], ['Harbhajan Singh', 2], ['JW Hastings', 3], ['TM Head', 0], ['MC Henriques', 2], ['GB Hogg', 2], ['JO Holder', 6

In [12]:

with open('finaldatabowl.csv',mode='w') as f:
    writer = csv.writer(f)
    for line in bowler_clusters:
        writer.writerow(line)
        